# Shor algorithm

This notebook is part of a set of notebooks on different quantum algorithms. The main source of these notes is: (Vedral, 2013). 

## Theory

### Shor's algorithm

Shor's algorithm is a method for factoring a natural number $N$. It relies on the existance of an efficient classical algorithm for finding the greatest common factor (gcf) (called Euclid's algorithm (Lynn, 2000)) and on an efficient method to deduce some random eigenstates of an operator (called quantum phase estimation).

The algorithm is (based on (Vedral, 2013, p. 140)):

1. Randomly generate some natural number $a$ where $2<a<N$.

2. If $a$ and $N$ are not coprime $\left(K = gcf(a,N) \neq 1\right)$, then the factorization of $N$ is $\frac{N}{K},K$ and the algorithm stopped.

3. If $a$ is odd, then the algorithm fails. Otherwise, deduce the order of $a$ (smallest $r$ where $a^r \bmod N = 1$) by the phase estimation method (see below).

4. Then, as $a^r -1 \bmod N = (a^\frac{r}{2} - 1)(a^\frac{r}{2} + 1) \bmod N  = 0$ and as $a^\frac{r}{2} \bmod N  = 1$ can't be true (by the definition of $r$), if $N$ doesn't divide $b = a^\frac{r}{2} + 1$ then $a$ and $b$ share common factors. These can be deduced by Euclid's algorithm. Meaning if $K = gcd(a,b) \neq 1$ then the factorization of $N$ is $\frac{N}{K},K$.

It can be shown that this process has a success probability of $\frac{1}{2}$ (Vedral, 2013, p. 142) and the complexity of the algorithm can be shown to be $O((\log{n})^2(\log{\log{n}})(\log{\log{\log{n}}}))$ (Shor, 1996, p. 15).

### Euclid's algorithm

Euclid's algorithm finds the gcf of two natural numbers (here these numbers shall be denoted by $a,b$ and it will be assumed that $a<b$). The algorithm works by recursively applying the formula $gcd(a,b) = gcd(a,b-a)$ until the gcd is trivial (Lynn, 2000).

### Phase estimation

This is the quantum part of the algorithm; it efficiently finds the order $r$ associated with $a$ and $N$. To find $r$, a qubit system of dimension $m >\log_2(N)$ is set up with the normal binary labelling convention used for the basis (eg. $\ket{0} = \ket{000}, \ket{1} = \ket{010}, \ket{2} = \ket{011},...$). An operator $\hat{U}_a$ is set up where $\hat{U}_a\ket{x} = \ket{ax \bmod N}$ (for efficient implementation see (Shor, 1996, p. 11)). It can then be shown that the eigenstates of $\hat{U}_a$ take the form $\lambda_n = e^{\frac{2\pi i n}{r}}$ and that $\ket{1}$ is proportional to the sum of all eigenkets (Vendral, 2013, p. 144). Due to these properties, the quantum phase estimation algorithm can then be applied to $\ket{1}$ (see (Shor, 1996, p. 15) and (Vendral, 2013, p. 142) for a description and circuit diagrams). The result is an equally weighted superposition corresponding to each of the different eigenstates. A measurement can then be made to find one of these eigenstates. The resulting number is approximately $2^{m}\frac{n}{r}$. $r$ can then be approximated by the denominator of the coprime fraction closest to $\frac{n}{r}$. This approximation can be made more accurate by performing the algorithm several times and taking the lowest common multiple (LCM) of the resulting r values.

## Implmentation

Below the algorithm is implemented. The result of the QM part of the algorithm is printed out and the classical part is then performed in a seperate function (the user inputs the results of the measurement of the QM register). Test cases were taken from: https://blendmaster.github.io/ShorJS/.



In [1]:
import numpy as np
import itertools
import sys
sys.path.append('C:\workspace\git-repos\physics-projects\quantum-algorithms')
%run register


Code for inverse QFT.

In [2]:
def invQFT(QFTRegister,start,end):
    Rs = []
    for i in range(2,end-start+1):
        Rs.append(np.array([[1,0],[0,np.exp(-2*np.pi*1j/(2**i),dtype=np.complex128)]],dtype=np.complex128))

    for i in reversed(range(end-start)):
        for j in reversed(range(end -start - i - 1)):
            gate = QFTRegister.findControlledSingleQubitOp(Rs[j], i+1+j, i)
            QFTRegister.applyGate(gate)

        QFTRegister.applyH(i)
    
    return QFTRegister


Test cases for inverse QFT.

In [3]:
#result should be 2
QFT1 = intialiseProductRegister([q0 + complex(0,1)*q1,q0-q1,q0+q1])
print(f"Initial state: {QFT1.displayQubits()}")
print(f"Result: {invQFT(QFT1,0,3).displayQubits()}")

Initial state: (0.354+0j)|0> + (0.354+0j)|1> + (-0.354+0j)|2> + (-0.354+0j)|3> + 0.354j|4> + 0.354j|5> + -0.354j|6> + -0.354j|7>
Result: (1+0j)|2>


In [4]:
#result should be 1
QFT2 = intialiseProductRegister([q0 + complex(0,1)*q1,q0-q1])
print(f"Initial state: {QFT2.displayQubits()}")
print(f"Result: {invQFT(QFT2,0,2).displayQubits()}")

Initial state: (0.5+0j)|0> + (-0.5+0j)|1> + 0.5j|2> + (-0-0.5j)|3>
Result: (1+0j)|1>


Code for phase estimation.

In [5]:
def findDominantBit(register, start, end):
    return int(register.binary[np.argmax(np.abs(register.qubits))][start:end],2)


def phaseEstimation(psiAcc,U,e):
    """
    Performs phase estimation.

    Inputs:
    psiAcc - accuracy of psi returned
    U - unitary matrix
    e - ket
    """
    eBits = np.log2(np.shape(e)[0]).astype(int)

    #better name
    outputRegister = intialiseProductRegister([q0 for i in range(psiAcc)]) 
    eigenvectorRegister = intialiseProductRegister([e])
    register = mergeRegisters(outputRegister,eigenvectorRegister) 
    
    initalStr = register.displayQubits(binBits=eBits,dType="intbin")
    print(f"Initial state (control qubits): {initalStr}")

    register.applySingleQubitProduct(np.array([[1,1],[1,-1]])/np.sqrt(2),0,psiAcc)

    HStr = register.displayQubits(binBits=eBits,dType="intbin") 
    print(f"H applied (control qubits): {HStr}")
    
    for i in range(psiAcc):
        gateP1 = mergeRegisterOperator(outputRegister.findSingleQubitOp(np.array([[1,0],[0,0]]),psiAcc-1-i),
                                        np.eye(eigenvectorRegister.N,dtype=np.complex128))
        gateP2 =  mergeRegisterOperator(outputRegister.findSingleQubitOp(np.array([[0,0],[0,1]]),psiAcc-1-i), 
                                        np.linalg.matrix_power(U,2**i))


        gate = gateP1 + gateP2
        register.applyGate(gate)

    HUStr = register.displayQubits(dType="intbin",binBits=eBits)
    print(f"H and Us applied (control qubits): {HUStr}")

    #this is required as the QFT must be applied to the control qubits in reverse order
    register.invertQubits(0,psiAcc)

    register = invQFT(register,0,psiAcc)

    invQFTStr = register.displayQubits(dType="intbin",binBits=eBits)
    print(f"IQFT (control qubits): {invQFTStr}") 
    #should display numbers calculated only up to psiAcc

    #The display bit isn't working properly, I want the final bits to not be included in the number, instead just being bit on the outside

    return register


Tests for phase estimation function.

In [ ]:
#result should be 0
U1 = np.array([[1,0],[0,1]])
e1 = np.array([1,0],dtype=np.complex128)
phaseEst1 = phaseEstimation(3,U1,e1)
bit1 = findDominantBit(phaseEst1,0,3)
print(f"Phase estimation result: {bit1/2**3}")

Initial state (control qubits): (1+0j)|0,0>
H applied (control qubits): (0.354+0j)|0,0> + (0.354+0j)|1,0> + (0.354+0j)|2,0> + (0.354+0j)|3,0> + (0.354+0j)|4,0> + (0.354+0j)|5,0> + (0.354+0j)|6,0> + (0.354+0j)|7,0>
H and Us applied (control qubits): (0.354+0j)|0,0> + (0.354+0j)|1,0> + (0.354+0j)|2,0> + (0.354+0j)|3,0> + (0.354+0j)|4,0> + (0.354+0j)|5,0> + (0.354+0j)|6,0> + (0.354+0j)|7,0>
IQFT (control qubits): (1+0j)|0,0>
Phase estimation result: 0.0


/Users/williamelderfield/git-rep/physics-projects/quantum-algorithms/register.py:65: RuntimeWarning: divide by zero encountered in matmul
  controlled = self.findSingleQubitOp(np.array([[0,0],[0,1]],dtype=np.complex128), controlIndex) @ self.findSingleQubitOp(op, targetIndex)
/Users/williamelderfield/git-rep/physics-projects/quantum-algorithms/register.py:65: RuntimeWarning: overflow encountered in matmul
  controlled = self.findSingleQubitOp(np.array([[0,0],[0,1]],dtype=np.complex128), controlIndex) @ self.findSingleQubitOp(op, targetIndex)
/Users/williamelderfield/git-rep/physics-projects/quantum-algorithms/register.py:65: RuntimeWarning: invalid value encountered in matmul
  controlled = self.findSingleQubitOp(np.array([[0,0],[0,1]],dtype=np.complex128), controlIndex) @ self.findSingleQubitOp(op, targetIndex)


In [7]:
#result should be 0.25
U2 = np.array([[complex(0,1),0],[0,1]]) 
e2 = np.array([1,0],dtype=np.complex128)
phaseEst2 = phaseEstimation(2,U2,e2)
bit2 = findDominantBit(phaseEst2,0,2)
print(f"Phase estimation result: {bit2/2**2}")

Initial state (control qubits): (1+0j)|0,0>
H applied (control qubits): (0.5+0j)|0,0> + (0.5+0j)|1,0> + (0.5+0j)|2,0> + (0.5+0j)|3,0>
H and Us applied (control qubits): (0.5+0j)|0,0> + 0.5j|1,0> + (-0.5+0j)|2,0> + -0.5j|3,0>
IQFT (control qubits): (1+0j)|1,0>
Phase estimation result: 0.25


In [8]:
#result shoudl be 0.5 
U3 = np.array([[np.exp(np.pi*complex(0,1)),0],[0,1]],dtype=np.complex128) 
e3 = np.array([1,0],dtype=np.complex128)
phaseEst3 = phaseEstimation(8,U3,e3)
bit2 = findDominantBit(phaseEst3,0,8)
print(f"Phase estimation result: {bit2/2**8}")

Initial state (control qubits): (1+0j)|0,0>
H applied (control qubits): (0.062+0j)|0,0> + (0.062+0j)|1,0> + (0.062+0j)|2,0> + (0.062+0j)|3,0> + (0.062+0j)|4,0> + (0.062+0j)|5,0> + (0.062+0j)|6,0> + (0.062+0j)|7,0> + (0.062+0j)|8,0> + (0.062+0j)|9,0> + ...
H and Us applied (control qubits): (0.062+0j)|0,0> + (-0.062+0j)|1,0> + (0.062-0j)|2,0> + (-0.062+0j)|3,0> + (0.062-0j)|4,0> + (-0.062+0j)|5,0> + (0.062-0j)|6,0> + (-0.062+0j)|7,0> + (0.062-0j)|8,0> + (-0.062+0j)|9,0> + (0.062-0j)|10,0> + (-0.062+0j)|11,0> + (0.062-0j)|12,0> + ...


/Users/williamelderfield/git-rep/physics-projects/quantum-algorithms/register.py:78: RuntimeWarning: divide by zero encountered in matmul
  self.qubits = self.getSingleQubitProduct(op,start,stop) @ self.qubits
/Users/williamelderfield/git-rep/physics-projects/quantum-algorithms/register.py:78: RuntimeWarning: overflow encountered in matmul
  self.qubits = self.getSingleQubitProduct(op,start,stop) @ self.qubits
/Users/williamelderfield/git-rep/physics-projects/quantum-algorithms/register.py:78: RuntimeWarning: invalid value encountered in matmul
  self.qubits = self.getSingleQubitProduct(op,start,stop) @ self.qubits
/Users/williamelderfield/git-rep/physics-projects/quantum-algorithms/register.py:82: RuntimeWarning: divide by zero encountered in matmul
  self.qubits = gate @ self.qubits
/Users/williamelderfield/git-rep/physics-projects/quantum-algorithms/register.py:82: RuntimeWarning: overflow encountered in matmul
  self.qubits = gate @ self.qubits
/Users/williamelderfield/git-rep/phys

IQFT (control qubits): (1-0j)|128,0>
Phase estimation result: 0.5


Code for full Shor algorithm.

In [ ]:
def findNearestDenominator(x):
    """
    Function that finds the two coprime numbers whose fraction is nearest to x.
    """
    xb = np.round(x)
    xa = x - xb
    if xa != 0:
        xaRecpb = np.round(1/xa)
        return xb*xaRecpb+1
    else:
        return xb
      

def shorQuantum(N,a,qNum,psiAcc):
    """
    Quantum part of algorithm that factorizes natural number N.

    Inputs:
    N - number being factorized
    a - int between 2 and N used in algorithm
    qNum - number of qubits used for U_a matrix
    psiAcc - accuracy of psi found in phase estimation
    """
    HDim = 2**qNum

    eigenvector1 = np.zeros([HDim],dtype=np.complex128)
    eigenvector1[1] = 1

    K = np.gcd(a,N)
    if K != 1 and K != N:
        print(f"Factorization is: {(N/K,K)}")
    
    #find U_a matrix
    U_a = np.zeros([HDim,HDim],dtype=np.complex128)
    for j,k in itertools.product(range(HDim), repeat=2): 
        if j == a*k % N and k < N:
            U_a[j,k] = 1
        elif j == k and k >= N:
            U_a[j,k] = 1
            
    phaseEstimation(psiAcc,U_a,eigenvector1)
    
def shorClassical(measurementResults,a,N,psiAcc):
    """
    Classical part of algorithm that factorizes natural number N.
    Inputs:
    measurementResults - list of integers found in quantum part of algorithm
    a - int between 2 and N used in algorithm
    N - number being factorized
    """

    if measurementResults != []:

        rs = [] #give r up to a common factor with n
        for m in measurementResults:
            rs.append(findNearestDenominator(2**psiAcc/m))
        R = np.lcm.reduce(np.array(rs,dtype=int))

        if R % 2 == 0:
            g = np.gcd(int(a**(R/2) + 1),int(N)) 
            if g != 1 and g != N:
                return N/g, g
    
    return "failed"


Tests for Shor algorithm.

In [ ]:
print("Shor's algorithm for factorizing 15")

shorQuantum(15,2,4,4)

print(f"The factorization is: {shorClassical([4,8,12],2,15,4)}")

Shor's algorithm for factorizing 15
Initial state (control qubits): (1+0j)|0,0001>
H applied (control qubits): (0.25+0j)|0,0001> + (0.25+0j)|1,0001> + (0.25+0j)|2,0001> + (0.25+0j)|3,0001> + (0.25+0j)|4,0001> + (0.25+0j)|5,0001> + (0.25+0j)|6,0001> + (0.25+0j)|7,0001> + (0.25+0j)|8,0001> + (0.25+0j)|9,0001> + ...
H and Us applied (control qubits): (0.25+0j)|0,0001> + (0.25+0j)|1,0010> + (0.25+0j)|2,0100> + (0.25+0j)|3,1000> + (0.25+0j)|4,0001> + (0.25+0j)|5,0010> + (0.25+0j)|6,0100> + (0.25+0j)|7,1000> + (0.25+0j)|8,0001> + (0.25+0j)|9,0010> + ...
IQFT (control qubits): (0.25+0j)|0,0001> + (0.25+0j)|0,0010> + (0.25+0j)|0,0100> + (0.25+0j)|0,1000> + (0.25+0j)|4,0001> + -0.25j|4,0010> + (-0.25+0j)|4,0100> + (-0+0.25j)|4,1000> + (0.25+0j)|8,0001> + (-0.25+0j)|8,0010> + (0.25+ ...
The factorization is: (np.float64(3.0), np.int64(5))


/Users/williamelderfield/git-rep/physics-projects/.venv/lib/python3.9/site-packages/numpy/linalg/_linalg.py:3220: RuntimeWarning: divide by zero encountered in matmul
  return _core_matmul(x1, x2)
/Users/williamelderfield/git-rep/physics-projects/.venv/lib/python3.9/site-packages/numpy/linalg/_linalg.py:3220: RuntimeWarning: overflow encountered in matmul
  return _core_matmul(x1, x2)
/Users/williamelderfield/git-rep/physics-projects/.venv/lib/python3.9/site-packages/numpy/linalg/_linalg.py:3220: RuntimeWarning: invalid value encountered in matmul
  return _core_matmul(x1, x2)


In [ ]:
print("Shor's algorithm for factorizing 12")

shorQuantum(12,5,4,8)

print(f"The factorization is: {shorClassical([128],5,12,8)}")

Shor's algorithm for factorizing 12
Initial state (control qubits): (1+0j)|0,0001>
H applied (control qubits): (0.062+0j)|0,0001> + (0.062+0j)|1,0001> + (0.062+0j)|2,0001> + (0.062+0j)|3,0001> + (0.062+0j)|4,0001> + (0.062+0j)|5,0001> + (0.062+0j)|6,0001> + (0.062+0j)|7,0001> + (0.062+0j)|8,0001> + (0.062+0j)|9,0001> + ...
H and Us applied (control qubits): (0.062+0j)|0,0001> + (0.062+0j)|1,0101> + (0.062+0j)|2,0001> + (0.062+0j)|3,0101> + (0.062+0j)|4,0001> + (0.062+0j)|5,0101> + (0.062+0j)|6,0001> + (0.062+0j)|7,0101> + (0.062+0j)|8,0001> + (0.062+0j)|9,0101> + ...
IQFT (control qubits): (0.5+0j)|0,0001> + (0.5+0j)|0,0101> + (0.5+0j)|128,0001> + (-0.5+0j)|128,0101>
The factorization is: (np.float64(2.0), np.int64(6))


In [ ]:
print("Shor's algorithm for factorizing 21")

shorQuantum(21,5,4,4)

print(f"The factorization is: {shorClassical([8],5,12,4)}")

Shor's algorithm for factorizing 21
Initial state (control qubits): (1+0j)|0,0001>
H applied (control qubits): (0.25+0j)|0,0001> + (0.25+0j)|1,0001> + (0.25+0j)|2,0001> + (0.25+0j)|3,0001> + (0.25+0j)|4,0001> + (0.25+0j)|5,0001> + (0.25+0j)|6,0001> + (0.25+0j)|7,0001> + (0.25+0j)|8,0001> + (0.25+0j)|9,0001> + ...
H and Us applied (control qubits): (0.25+0j)|0,0001> + (0.25+0j)|1,0101> + (0.25+0j)|2,0100>
IQFT (control qubits): (0.062+0j)|0,0001> + (0.062+0j)|0,0100> + (0.062+0j)|0,0101> + (0.062+0j)|1,0001> + (0.044-0.044j)|1,0100> + (0.058-0.024j)|1,0101> + (0.062+0j)|2,0001> + -0.062j|2,0100> + (0.044-0.044j)|2,0101> + (0.062+0j)|3,0001> + (-0.044-0.044j)|3,0100> + (0.024-0.058j)|3,0101> + (0.062+0j)|4,0001> + ...
The factorization is: (np.float64(2.0), np.int64(6))


# References

Vedral, V., 2013. Introduction to Quantum Information Science. Oxford: Oxford University Press.

Lynn, B., 2000. Euclid's Algorithm. [Online] 
Available at: https://crypto.stanford.edu/pbc/notes/numbertheory/euclid.html
[Accessed 27 August 2024].

Shor, P., 1996. Polynomial-Time Algorithms for Prime Factorization and Discrete Logarithms on a Quantum Computer. 



